In [1]:
#This notebook was originally copied from https://www.kaggle.com/samiaimad/ai-uni-project. 
#This copy is to make changes of my own 

#using virtual environment Voice

#Rogier Landman

import pandas as pd
import numpy as np
import os
import sys

# librosa is a Python library for analyzing audio and music.
import librosa
import librosa.display
import seaborn as sns
import matplotlib.pyplot as plt
import sklearn

from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.metrics import confusion_matrix, classification_report
from sklearn.model_selection import train_test_split

# to play the audio files
from IPython.display import Audio

import keras
from keras.callbacks import ReduceLROnPlateau
from keras.models import Sequential
from keras.layers import Dense, Conv1D, MaxPooling1D, Flatten, Dropout, BatchNormalization
from keras.utils import np_utils
from keras.callbacks import ModelCheckpoint

import wave
import pylab
from pathlib import Path
from scipy import signal
from scipy.io import wavfile
from sklearn.metrics import confusion_matrix
import itertools
import pylab

import matplotlib.pyplot as plt
import math

import tensorflow as tf
from tensorflow.keras.datasets import mnist
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D
from tensorflow.keras.layers import MaxPool2D
from tensorflow.keras.layers import Flatten
from tensorflow.keras.layers import Dropout
from tensorflow.keras.layers import Dense

import warnings
if not sys.warnoptions:
    warnings.simplefilter("ignore")
warnings.filterwarnings("ignore", category=DeprecationWarning)

# **Assigning Data Paths to Variables**

In [2]:
Ravdess = "/kaggle/input/ravdess-emotional-speech-audio/audio_speech_actors_01-24/"
CremaD = "/kaggle/input/cremad/AudioWAV/"
Tess = "/kaggle/input/toronto-emotional-speech-set-tess/tess toronto emotional speech set data/TESS Toronto emotional speech set data/"
Savee = "/kaggle/input/savee-database/AudioData"


# **Audio Classification from the ravdess-emotional-speech-audio data set**

In [3]:
ravdess=os.listdir(Ravdess)

emotion=[]
path=[]
gender=[]
intensity=[]
for i in ravdess:
    actor=os.listdir(Ravdess+i)
    for f in actor:
        fsplit= f.split('.')[0]
        fsplit=fsplit.split('-')
        emotion.append(int(fsplit[2]))
        gend=int(fsplit[6])
        if gend%2==0:
            gender.append('Female')
        else:
            gender.append('Male')
        intensity.append(int(fsplit[3]))
        for sp in emotion:
            if sp=="1":
                intensity.append('Normal')
        path.append(Ravdess + i + '/' + f)
edf=pd.DataFrame(emotion, columns=['Emotions'])
pdf=pd.DataFrame(path, columns=['Path'])
gdf=pd.DataFrame(gender, columns=['Gender'])
idf=pd.DataFrame(intensity, columns=['Intensity'])

ravdess_df=pd.concat([edf, pdf, gdf, idf], axis=1)

ravdess_df.Emotions.replace({1:'neutral', 2:'calm', 3:'happy', 4:'sad', 5:'angry', 6:'fear', 7:'disgust', 8:'surprise'}, inplace=True)
ravdess_df.Intensity.replace({1:'Normal', 2:'Strong'}, inplace=True)
ravdess_df.head()
ravdess_df.isnull()
ravdess_df.info()

# **Audio Classification from the cremad data set**

In [4]:
crema=os.listdir(CremaD)

emotion=[]
path=[]
gender=[]
intensity=[]
for f in crema:
    part2= f.split('.')[0]
    part2=part2.split('_')
    
    if int(part2[0])%2==0:
        gender.append('Female')
    else:
        gender.append('Male')
        
    if part2[2] == 'SAD':
        emotion.append('sad')
    elif part2[2] == 'ANG':
        emotion.append('angry')
    elif part2[2] == 'DIS':
        emotion.append('disgust')
    elif part2[2] == 'FEA':
        emotion.append('fear')
    elif part2[2] == 'HAP':
        emotion.append('happy')
    elif part2[2] == 'NEU':
        emotion.append('neutral')
    else:
        emotion.append('Unknown')
        
    if part2[3]=='HI':
        intensity.append('High')
    elif part2[3]=='LO':
        intensity.append('Low')
    elif part2[3]=='MD':
        intensity.append('Medium')
    elif part2[3]=='XX':
        intensity.append('Unspecified')
        
    path.append(CremaD+f)
    
edf=pd.DataFrame(emotion, columns=['Emotions'])
pdf=pd.DataFrame(path, columns=['Path'])
gdf=pd.DataFrame(gender, columns=['Gender'])
idf=pd.DataFrame(intensity, columns=['Intensity'])

crema_df=pd.concat([edf, pdf, gdf, idf], axis=1)
crema_df.head()
crema_df.info()

# **Audio Classification from the toronto-emotional-speech-set-tess data set**

In [5]:
tess=os.listdir(Tess)

emotion=[]
path=[]
gender=[]
intensity=[]

for i in tess:
    dir=os.listdir(Tess+i)
    for f in dir:
        fsplit= f.split('.')[0]
        fsplit=fsplit.split('_')[2]
        if fsplit=='ps':
            emotion.append('surprise')
        else:
            emotion.append(fsplit)
        gender.append('Female')
        path.append(Tess+i+'/'+f)
        intensity.append('Unspecified')
        
edf=pd.DataFrame(emotion, columns=['Emotions'])
pdf=pd.DataFrame(path, columns=['Path'])
gdf=pd.DataFrame(gender, columns=['Gender'])
idf=pd.DataFrame(intensity, columns=['Intensity'])

tess_df=pd.concat([edf, pdf, gdf, idf], axis=1)
tess_df.head()
tess_df.isnull()
tess_df.info()

# **Audio Classification from Savee Dataset**

In [6]:
import os
import re
Savee = "/kaggle/input/savee-database/AudioData"
savee = os.listdir(Savee)

emotion=[]
path=[]
gender=[]
intensity=[]

for f in savee:
    if(f!="Info.txt"):
        x= Savee+"/"+f
        dir=os.listdir(x)
    
    for d in dir:
        dsplit = d.split(".")[0]
        llsplit=re.split('\d+',dsplit[0])
        #print(llsplit)

        for l in llsplit:
            elist=l
            if elist=='a':
                emotion.append('angry')
            elif elist=='d':
                emotion.append('disgust')
            elif elist=='f':
                emotion.append('fear')
            elif elist=='h':
                emotion.append('happy')
            elif elist=='n':
                emotion.append('neutral')
            elif elist=='sa':
                emotion.append('sad')
            else:
                emotion.append('surprise')
        gender.append('Male')
        path.append(Savee+"/"+f)
        intensity.append('Unspecified')

edf=pd.DataFrame(emotion, columns=['Emotions'])
pdf=pd.DataFrame(path, columns=['Path'])
gdf=pd.DataFrame(gender, columns=['Gender'])
idf=pd.DataFrame(intensity, columns=['Intensity'])

savee_df=pd.concat([edf, pdf, gdf, idf], axis=1)
#savee_df.head()
savee_df.isnull()
savee_df.info()
savee_df.tail()


# **Combining DataFrames (ravdess_df, crema_df,tess_df, savee_df)**

In [7]:
Data_Combined = pd.concat([ravdess_df, crema_df,tess_df, savee_df], axis = 0)
Data_Combined.isnull()
Data_Combined.info()

# **Data Visualization**

In [8]:
x = []
flag = 0

while (flag != 100): 
   
    audio = wave.open(Data_Combined.iloc[flag, 1], 'r')
    flag = flag + 1 
    y= Data_Combined.iloc[flag, 0]
    
   
    if (y not in x):
        signal = np.frombuffer(audio.readframes(-1), dtype=np.int16)
        plt.figure(figsize=(12,12))
        plot1 = plt.subplot(211)
        plot1.set_title(Data_Combined.iloc[flag, 0])
        plot1.plot(signal)
        plot1.set_xlabel('time * Number of samples')
        plot1.set_ylabel('energy')

        plot2 = plt.subplot(212)
        plot2.specgram(signal, NFFT=1024, Fs=12281, noverlap=900)
        plot2.set_xlabel('Time')
        plot2.set_ylabel('Frequency')
        
        x.append(y)
    
    if (len(x)==8):
        flag=100

plt.show()

# **Data Augmentation**

In [9]:
def stretch(data, rate=0.8):
    return librosa.effects.time_stretch(data, rate)
#x , sr = librosa.load(stretch(audio_data))
#ya = stretch(x)
#plt.figure(figsize=(14,4))
#librosa.display.waveplot(y=ya, sr=sr)
#Audio(ya, rate=sr)

#SNR = 20*log(RMS_s/RMS_n)


In [10]:
def white_noise(data,SNR):
    RMS_s=math.sqrt(np.mean(data**2))
    RMS_n=math.sqrt(RMS_s**2/(pow(10,SNR/10)))
    STD_n=RMS_n
    noise=np.random.normal(0, STD_n, data.shape[0])
    data_noise=noise+data
    return data_noise 
    

x , sr = librosa.load(audio_data)
ya = white_noise(x,SNR=10)
plt.figure(figsize=(14,4))
librosa.display.waveplot(y=ya, sr=sr)
Audio(ya, rate=sr)

In [ ]:
def shifting_time(data, sampling_rate, shift_max, shift_direction):
    shift = np.random.randint(sampling_rate * shift_max)
    if shift_direction == 'right':
        shift = -shift
    elif self.shift_direction == 'both':
        direction = np.random.randint(0, 2)
        if direction == 1:
            shift = -shift
    augmented_data = np.roll(data, shift)
    # Set to silence for heading/ tailing
    if shift > 0:
        augmented_data[:shift] = 0
    else:
        augmented_data[shift:] = 0
    return augmented_data

x , sr = librosa.load(audio_data)
ya = shifting_time(x,sr,shift_max=1.5,shift_direction='right')
plt.figure(figsize=(14,4))
librosa.display.waveplot(y=ya, sr=sr)
Audio(ya, rate=sr)

In [ ]:
def pitch(data, sampling_rate, pitch_factor=0.7):
    return librosa.effects.pitch_shift(data, sampling_rate, pitch_factor)
x , sr = librosa.load(audio_data)
ya = pitch(x,sr,pitch_factor=0.5)
plt.figure(figsize=(14,4))
librosa.display.waveplot(y=ya, sr=sr)
Audio(ya, rate=sr)

# **Creating Spectograms images from Audio files**

In [ ]:
from pathlib import Path
import pylab

output_path="./"
if  not os.path.exists(os.path.join(output_path, 'audio-spectrograms')):
    os.mkdir(os.path.join(output_path, 'audio-spectrograms'))
    
#function to get sound and frame rate info
def get_audio_info(audio_path):
    wav = wave.open(audio_path, 'r')
    frames = wav.readframes(-1)
    audio_info = pylab.frombuffer(frames, 'int16')
    frame_rate = wav.getframerate()
    wav.close()
    return audio_info, frame_rate



for f in range (len(Data_Combined)):
        file_Path = f'{Data_Combined.iloc[f, 1]}'
        #print (file_path)
        file_stem = Path(file_Path).stem
        #print (file_stem)
        target_dir = f'class_{Data_Combined.iloc[f, 0]}'
        #print (target_dir)
        dist_dir = os.path.join(os.path.join(output_path, 'audio-spectrograms'), target_dir)
        #print (dist_dir)
        # ex: ./audio-spectrograms/class_fear
        file_dist_path = os.path.join(dist_dir, file_stem)
        #ex: ./audio-spectrograms/class_fear/03-01-06-02-02-01-10
        
    
        if not os.path.exists(file_dist_path + '.png'):
            if not os.path.exists(dist_dir):
                os.mkdir(dist_dir)
            file_stem = Path(file_Path).stem
            audio_info, frame_rate = get_audio_info(file_Path)
            pylab.specgram(audio_info, Fs=frame_rate)
            pylab.savefig(f'{file_dist_path}.png')
            pylab.close()

# **Data Preparation**

In [ ]:
# Declare constants
image_h = 256
image_w = 256
batch_size = 32
num_channel = 3
num_classes = 10
OUTPUT_DIR = '/kaggle/working/'

# dataset training spectrograms
training_dataset = tf.keras.preprocessing.image_dataset_from_directory(
                                             batch_size=batch_size,
                                             validation_split=0.2,
                                             directory=os.path.join(OUTPUT_DIR, 'audio-spectrograms'),
                                             shuffle=True,
                                             color_mode='rgb',
                                             image_size=(image_h, image_w),
                                             subset="training",
                                             seed=0)

# dataset validation spectrogram
validation_dataset = tf.keras.preprocessing.image_dataset_from_directory(
                                             batch_size=batch_size,
                                             validation_split=0.2,
                                             directory=os.path.join(OUTPUT_DIR, 'audio-spectrograms'),
                                             shuffle=True,
                                             color_mode='rgb',
                                             image_size=(image_h, image_w),
                                             subset="validation",
                                             seed=0)

In [ ]:
def prepare(ds):
    # Define our one transformation
    rescale = tf.keras.Sequential([tf.keras.layers.experimental.preprocessing.Rescaling(1./255)])
    ds = ds.map(lambda x, y: (rescale(x, training=True), y))
    return ds

training_dataset = prepare(training_dataset)
validation_dataset = prepare(validation_dataset)

# **Modeling**

In [ ]:
model=tf.keras.models.Sequential()
model.add(tf.keras.layers.Input(shape=(image_h,image_w,num_channel)))
#convolution layer
model.add(tf.keras.layers.Conv2D(32,3,strides=2,padding='same',activation='relu'))
model.add(tf.keras.layers.BatchNormalization())
#pooling layer
model.add(tf.keras.layers.MaxPooling2D(pool_size=(2,2)))
model.add(tf.keras.layers.BatchNormalization())
model.add(tf.keras.layers.Conv2D(64, 3, padding='same', activation='relu'))
model.add(tf.keras.layers.BatchNormalization())
model.add(tf.keras.layers.MaxPooling2D(pool_size=(2,2)))
model.add(tf.keras.layers.BatchNormalization())
model.add(tf.keras.layers.Conv2D(128, 3, padding='same', activation='relu'))
model.add(tf.keras.layers.BatchNormalization())
model.add(tf.keras.layers.MaxPooling2D(pool_size=(2,2)))
model.add(tf.keras.layers.BatchNormalization())
#fully connected layer
model.add(tf.keras.layers.Flatten())
model.add(tf.keras.layers.Dense(256, activation='relu'))
model.add(tf.keras.layers.BatchNormalization())
model.add(tf.keras.layers.Dropout(0.5))
#output layer
model.add(tf.keras.layers.Dense(10, activation='softmax'))

model.compile(loss='sparse_categorical_crossentropy', optimizer=tf.keras.optimizers.RMSprop(),metrics=['accuracy'],)

history=model.fit(training_dataset, epoch=10, validation=validation_dataset)

In [ ]:
#loss graph for training and validation
history=history.history
loss=history['loss']
values=history['value']
epochs=range(1,len(loss)+1)

plt.figure(figsize=(8,6))
plt.plot(epochs, loss, 'bo', label="Training loss")
plt.plot(epochs, values, 'b', label='Validation loss')
plt.title('Training and validation loss')
plt.xlabel('Epochs')
plt.ylabel('Loss')
plt.legend()
plt.show()

In [ ]:
final_loss, final_acc=model.evaluate(valid, verbose=0)
print("Final loss: {0:.6f}\n Final accuracy: {1:.6f}".format(final_loss, final_acc))